In [4]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

In [5]:
def get_ticker_data(stock: str, start: datetime = None, end: datetime = None):
    beg = datetime(1970, 1, 1)
    if start == None:
        start = beg
    if end == None:
        end = datetime.now()

    sdate = int((start - beg).total_seconds())
    edate = int((end - beg).total_seconds())

    url = f'https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={sdate}&period2={edate}&interval=1d&events=history&includeAdjustedClose=true'
    return pd.read_csv(url, parse_dates=True)

In [31]:
msft = get_ticker_data('MSFT')
msft['Date'] = msft['Date'].apply(pd.to_datetime)
#msft_series = msft[['Date', 'Close']]
items = msft.rolling(5, min_periods=5)
x = 0
for i in items:
    print(i)
    x+=1
    if x > 7:
        break

028  0.421875  0.430556   0.273468  66283200
567 1988-06-09  0.432292  0.436632  0.427951  0.428819   0.272365  69264000
568 1988-06-10  0.428819  0.435764  0.425347  0.432292   0.274571  45014400
569 1988-06-13  0.434028  0.437500  0.432292  0.433160   0.275122  31118400
          Date      Open      High       Low     Close  Adj Close    Volume
566 1988-06-08  0.421875  0.434028  0.421875  0.430556   0.273468  66283200
567 1988-06-09  0.432292  0.436632  0.427951  0.428819   0.272365  69264000
568 1988-06-10  0.428819  0.435764  0.425347  0.432292   0.274571  45014400
569 1988-06-13  0.434028  0.437500  0.432292  0.433160   0.275122  31118400
570 1988-06-14  0.437500  0.440972  0.435764  0.437500   0.277878  60379200
          Date      Open      High       Low     Close  Adj Close    Volume
567 1988-06-09  0.432292  0.436632  0.427951  0.428819   0.272365  69264000
568 1988-06-10  0.428819  0.435764  0.425347  0.432292   0.274571  45014400
569 1988-06-13  0.434028  0.437500  0.43229

KeyboardInterrupt: 

In [ ]:
smin, smax = msft[['Close']].min(), msft[['Close']].max()

In [ ]:
mclose = msft[['Close']]
scaled = (mclose-smin)/(smax-smin)

In [ ]:
def split_data(stock_val, lookback):
    data_raw = stock_val.values # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data)
    test_set_size = int(np.round(0.2*data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_train = torch.from_numpy(y_train).type(torch.Tensor)
    y_test = torch.from_numpy(y_test).type(torch.Tensor)
    
    return [x_train, y_train, x_test, y_test]

In [ ]:
xt, yt, xe, ye = split_data(mclose, 5)